In [1]:
import time
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, create_engine, Float
from sqlalchemy.orm import scoped_session, sessionmaker
import pandas as pd
from sqlalchemy import Table
import os

Base = declarative_base()
DBSession = scoped_session(sessionmaker())
engine = None

'''
class WeatherSFTable(Base):
    __tablename__ = 'eimuTable'
    id = Column(Integer, primary_key=True)
    Unnamed = Column(Integer, primary_key=False)
    expire_date_time = Column(Integer, primary_key=False)    
    valid_date_time = Column(Integer, primary_key=False)    
    temp = Column(Float, primary_key=False)    
    wx_phrase = Column(String, primary_key=False)    
    heat_index = Column(Float, primary_key=False)    
    rh = Column(Float, primary_key=False)    
    pressure = Column(Float, primary_key=False)    
    vis = Column(Float, primary_key=False)    
    wc = Column(Float, primary_key=False)    
    wdir = Column(Float, primary_key=False)    
    wspd = Column(Float, primary_key=False)    
    max_temp = Column(Float, primary_key=False)    
    min_temp = Column(Float, primary_key=False)
    expire_date_time = Column(String, primary_key=False)
    valid_date_time = Column(String, primary_key=False)  
    # Define columns matching your CSV data types
    # Example: name = Column(String)
    
'''
def init_sqlalchemy(dbname='sqlite:///sf.weather.db'):
    global engine
    engine = create_engine(dbname, echo=False)
    DBSession.remove()
    DBSession.configure(bind=engine, autoflush=False, expire_on_commit=False)
    Base.metadata.drop_all(engine)
    Base.metadata.create_all(engine)

def load_data_from_csv(file_path):
    data = pd.read_csv(file_path)
    return data

def create_table_from_csv(file_path):
    data = load_data_from_csv(file_path)
    metadata = Base.metadata
    print('hello')
    table_name = "WeatherSFTable"  # Specify the table name
    WeatherSFTable = Table(
        table_name,
        metadata,
        Column('id', Integer, primary_key=True)
    )

    for column_name, dtype in data.dtypes.items():
        if dtype == 'int64':
            column = Column(column_name, Integer)
        elif dtype == 'float64':
            column = Column(column_name, Float)
        else:
            column = Column(column_name, String)
        WeatherSFTable.append_column(column)

    metadata.create_all(engine)
    return WeatherSFTable



def bulk_insert_data(table, data):
    DBSession.bulk_insert_mappings(table, data)
    DBSession.commit()

if __name__ == '__main__':
    init_sqlalchemy()
    
    # Load data from CSV (replace 'your_data.csv' with your dataset file)
    data = load_data_from_csv('C://Study//CSE 4000//New folder//Thesis//dataset//weather_data_sf_2004_2023.csv')
    
    # Create the table dynamically
    table = create_table_from_csv('C://Study//CSE 4000//New folder//Thesis//dataset//weather_data_sf_2004_2023.csv')
    class EimuModel(Base):
        __table__ = table
    # Perform bulk insert
    bulk_insert_data(EimuModel, data.to_dict(orient='records'))

C:\Users\Resam Zaha\AppData\Local\Temp\ipykernel_11788\4064411039.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


hello
